# Pandas for Exploratory Data Analysis II 

Pandas is a very useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.
- Concatenating objects with `.append()` and `.concat()`
- Combining objects with `.join()` and `.merge()`
- Combining timeseries objects with `.merge_ordered()`


We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the AdventureWorks Cycles Dataset?
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


### Import Pandas

In [1]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling amongst other things
import seaborn as sns # plotting library
import matplotlib.pyplot as plt # plotting library
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and the `\t` separator to specify tab-delimited columns).

In [7]:
# Read in the dataset

prod = pd.read_csv('Production.Product.csv', sep='\t')

In [4]:
# Let's check out the first 3 rows again, for old time's sake

prod.head(3)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000


In [5]:
# And the number of rows x cols

prod.shape

(504, 25)

### Reset our index (like last time)

Let's bring our `ProductID` column into the index since it's the PK (primary key) of our table and that's where PKs belong as a best practice.

In [8]:
# Replace the auto-generated index with the ProductID column

prod.set_index('ProductID',inplace=True)

In [9]:
prod.head(3)

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000


## Handling missing data

Recall missing data is a systemic, challenging problem for data analysts and data scientists. Imagine conducting a poll, but some of the data gets lost, or you run out of budget and can't complete it! 😮<br><br>

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data (known in the business as `imputation` or `imputing`)
- The act of filling in missing data with Pandas


### Identifying missing data

Before *handling*, we must identify we're missing data at all!

We have a few ways to explore missing data, and they are reminiscent of our Boolean filters.

In [10]:
# True when data isn't missing
prod.notnull()

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,True,True,True,True,False,True,True,True,True,False,...,False,False,False,False,False,True,False,False,True,True
2,True,True,True,True,False,True,True,True,True,False,...,False,False,False,False,False,True,False,False,True,True
3,True,True,True,True,False,True,True,True,True,False,...,False,False,False,False,False,True,False,False,True,True
4,True,True,True,True,False,True,True,True,True,False,...,False,False,False,False,False,True,False,False,True,True
316,True,True,True,True,False,True,True,True,True,False,...,False,False,False,False,False,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,True,True,True,False,True,True,True,True,False,...,False,True,False,True,True,True,False,False,True,True
996,True,True,True,True,False,True,True,True,True,False,...,False,True,False,True,True,True,False,False,True,True
997,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,True,True


In [11]:
# True when data is missing
prod.isnull()

,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
2,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
3,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
4,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
316,False,False,False,False,True,False,False,False,False,True,...,True,True,True,True,True,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,True,False,False,False,False,True,...,True,False,True,False,False,False,True,True,False,False
996,False,False,False,False,True,False,False,False,False,True,...,True,False,True,False,False,False,True,True,False,False
997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False


Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [13]:
# Here is a quick and dirty way to do it
prod.isnull().sum()[prod.isnull().sum() !=0]

Color                    248
Size                     293
SizeUnitMeasureCode      328
WeightUnitMeasureCode    299
Weight                   299
ProductLine              226
Class                    257
Style                    293
ProductSubcategoryID     209
ProductModelID           209
SellEndDate              406
DiscontinuedDate         504
dtype: int64

Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing. Here is another implementation of `prod.isnull().sum()`, only wrapped with a `DataFrame` and some labels to make it a little more user-friendly:

In [14]:
# Or we can make things pretty as follows
null_df = pd.DataFrame(prod.isnull().sum(), columns=['Count of Nulls'])
null_df.index.name = 'Column'
null_df.sort_values(['Count of Nulls'], ascending=False).head(10)

,Count of Nulls
Column,
DiscontinuedDate,504
SellEndDate,406
SizeUnitMeasureCode,328
Weight,299
WeightUnitMeasureCode,299
Size,293
Style,293
Class,257
Color,248


### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The mean of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Follow up with the authority providing data that is missing


In our case, let's focus on handling missing values in `Color`. Let's get a count of the unique values in that column. We will need to use the `dropna=False` kwarg, otherwise the `pd.Series.value_counts()` method will not count `NaN` (null) values.

In [15]:
# Let's get a value count with the nulls included
prod['Color'].value_counts(dropna=False)

NaN             248
Black            93
Silver           43
Red              38
Yellow           36
Blue             26
Multi             8
Silver/Black      7
White             4
Grey              1
Name: Color, dtype: int64

In [16]:
prod['Color'].value_counts(dropna=True)

Black           93
Silver          43
Red             38
Yellow          36
Blue            26
Multi            8
Silver/Black     7
White            4
Grey             1
Name: Color, dtype: int64

In [18]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504 entries, 1 to 999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Name                   504 non-null    object 
 1   ProductNumber          504 non-null    object 
 2   MakeFlag               504 non-null    int64  
 3   FinishedGoodsFlag      504 non-null    int64  
 4   Color                  256 non-null    object 
 5   SafetyStockLevel       504 non-null    int64  
 6   ReorderPoint           504 non-null    int64  
 7   StandardCost           504 non-null    float64
 8   ListPrice              504 non-null    float64
 9   Size                   211 non-null    object 
 10  SizeUnitMeasureCode    176 non-null    object 
 11  WeightUnitMeasureCode  205 non-null    object 
 12  Weight                 205 non-null    float64
 13  DaysToManufacture      504 non-null    int64  
 14  ProductLine            278 non-null    object 
 15  Class 

Ahoy! We have 248 nulls!

Option 1: Drop the missing values.

In [21]:
# Drops rows where any row has a missing value - this does not happen *in place*, 
# so we are not actually dropping

x=prod['Color'].dropna(inplace=False)

In [22]:
x.shape

(256,)

In [1]:
prod.dropna(subset=['DiscontinuedDate'],axis=1)

NameError: name 'prod' is not defined

In [ ]:
prod.dropna

In [30]:
i=prod.dropna(how='all')

In [31]:
i.columns

Index(['Name', 'ProductNumber', 'MakeFlag', 'FinishedGoodsFlag', 'Color',
       'SafetyStockLevel', 'ReorderPoint', 'StandardCost', 'ListPrice', 'Size',
       'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight',
       'DaysToManufacture', 'ProductLine', 'Class', 'Style',
       'ProductSubcategoryID', 'ProductModelID', 'SellStartDate',
       'SellEndDate', 'DiscontinuedDate', 'rowguid', 'ModifiedDate'],
      dtype='object', name='Column')

**Important!** `pd.DataFrame.dropna()` and `pd.Series.dropna()` are very versatile! Let's look at the docs (Series is similar):

```python
Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. deprecated:: 0.23.0: Pass tuple or list to drop on multiple
    axes.
how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.
thresh : int, optional
    Require that many non-NA values.
subset : array-like, optional
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include.
inplace : bool, default False
    If True, do operation inplace and return None.
```

**how**: This tells us if we want to remove a row if _any_ of the columns have a null, or _all_ of the columns have a null.<br>
**subset**: We can input an array here, like `['Color', 'Size', 'Weight']`, and it will only consider nulls in those columns. This is very useful!<br>
**inplace**: This is if you want to mutate (change) the source dataframe. Default is `False`, so it will return a _copy_ of the source dataframe.

To accomplish the same thing, but implement it on our entire dataframe, we can do the following:

In [24]:
# Drops all nulls from the Color column, but returns the entire dataframe 
# instead of just the Color column
prod.dropna(how='any', subset=['Color'], inplace=False).shape

(256, 24)

Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or mode (most frequently occurring). For `Color`, let's replace the nulls with the string value `NoColor`.

Let's first look at the way we'd do it with a single column, using the `pd.Series.fillna()` method:

In [34]:
prod['Color'].fillna(value='NoColor').head(10)

ProductID
1      NoColor
2      NoColor
3      NoColor
4      NoColor
316    NoColor
317      Black
318      Black
319      Black
320     Silver
321     Silver
Name: Color, dtype: object

In [33]:
prod['Color'].value_counts()

Black           93
Silver          43
Red             38
Yellow          36
Blue            26
Multi            8
Silver/Black     7
White            4
Grey             1
Name: Color, dtype: int64

Now let's see how we'd do it to the whole dataframe, using the `pd.DataFrame.fillna()` method. Notice the similar API between the methods with the `value` kwarg. Good congruent design, pandas development team! The full dataframe is returned, and not just a column.

In [35]:
prod.fillna(value={'Color': 'NoColor'}).head(10)

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,Adjustable Race,AR-5381,0,0,NoColor,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
2,Bearing Ball,BA-8327,0,0,NoColor,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
3,BB Ball Bearing,BE-2349,1,0,NoColor,800,600,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000
4,Headset Ball Bearings,BE-2908,0,0,NoColor,800,600,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B},2014-02-08 10:01:36.827000000
316,Blade,BL-2036,1,0,NoColor,800,600,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{E73E9750-603B-4131-89F5-3DD15ED5FF80},2014-02-08 10:01:36.827000000
317,LL Crankarm,CA-5965,0,0,Black,500,375,0.0,0.0,NaN,...,NaN,L,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{3C9D10B7-A6B2-4774-9963-C19DCEE72FEA},2014-02-08 10:01:36.827000000
318,ML Crankarm,CA-6738,0,0,Black,500,375,0.0,0.0,NaN,...,NaN,M,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{EABB9A92-FA07-4EAB-8955-F0517B4A4CA7},2014-02-08 10:01:36.827000000
319,HL Crankarm,CA-7457,0,0,Black,500,375,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7D3FD384-4F29-484B-86FA-4206E276FE58},2014-02-08 10:01:36.827000000
320,Chainring Bolts,CB-2903,0,0,Silver,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{7BE38E48-B7D6-4486-888E-F53C26735101},2014-02-08 10:01:36.827000000


But wait! There's more! We can reference any other data or formulas we want with the imputation (the value we fill the nulls with). This is very handy if you want to impute with the average or median of that column... or even another column altogether! 

Here is an example where we will the nulls of `Color` with the average value from the `ListPrice` column. This has no practical value in this application, but immense value in other applications.

In [36]:
prod.fillna(value={'Color': prod['ListPrice'].mean()}).head(3)

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
ProductID,,,,,,,,,,,,,,,,,,,,,
1,Adjustable Race,AR-5381,0,0,438.666,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{694215B7-08F7-4C0D-ACB1-D734BA44C0C8},2014-02-08 10:01:36.827000000
2,Bearing Ball,BA-8327,0,0,438.666,1000,750,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{58AE3C20-4F3A-4749-A7D4-D568806CC537},2014-02-08 10:01:36.827000000
3,BB Ball Bearing,BE-2349,1,0,438.666,800,600,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,2008-04-30 00:00:00,NaN,NaN,{9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E},2014-02-08 10:01:36.827000000


They're gone! Important points:

- Don't forget to use the `inplace=True` kwarg to mutate the source dataframe (i.e. `save changes`). 
- It is helpful to not use `inplace=True` initially to ensure your code/logic is correct, prior to making permanent changes.

## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute, for example, group by `Color`
- **Combine:** Put our DataFrame back together and return some _aggregated_ metric, such as the `sum`, `count`, or `max`.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

Let's group by `Color`, and get a count of products for each color.

In [41]:
# Group by Color, giving the number of products of each color

prod.groupby(['Color','Style']).count()

Column        Name  ProductNumber  MakeFlag  FinishedGoodsFlag  \
Color  Style                                                     
Black  M         4              4         4                  4   
       U        61             61        61                 61   
       W         6              6         6                  6   
Blue   U        25             25        25                 25   
Multi  M         3              3         3                  3   
       U         5              5         5                  5   
Red    U        37             37        37                 37   
Silver U        22             22        22                 22   
       W         8              8         8                  8   
White  U         4              4         4                  4   
Yellow U        22             22        22                 22   
       W        14             14        14                 14   

Column        SafetyStockLevel  ReorderPoint  StandardCost  ListPrice  Size  \
Color  Style                                                                  
Black  M                     4             4             4          4     4   
       U                    61            61            61         61    61   
       W                     6             6             6          6     6   
Blue   U                    25            25            25         25    25   
Multi  M                     3             3             3          3     3   
       U                     5             5             5          5     4   
Red    U                    37            37            37         37    37   
Silver U                    22            22            22         22    22   
       W                     8             8             8          8     8   
White  U                     4             4             4          4     4   
Yellow U                    22            22            22         22    22   
       W                    14            14            14         14    14   

Column        SizeUnitMeasureCode  ...  DaysToManufacture  ProductLine  Class  \
Color  Style                       ...                                          
Black  M                        0  ...                  4            4      0   
       U                       55  ...                 61           61     55   
       W                        0  ...                  6            6      0   
Blue   U                       22  ...                 25           25     22   
Multi  M                        0  ...                  3            3      0   
       U                        0  ...                  5            5      0   
Red    U                       37  ...                 37           37     37   
Silver U                       22  ...                 22           22     22   
       W                        8  ...                  8            8      8   
White  U                        0  ...                  4            4      0   
Yellow U                       18  ...                 22           22     18   
       W                       14  ...                 14           14     14   

Column        ProductSubcategoryID  ProductModelID  SellStartDate  \
Color  Style                                                        
Black  M                         4               4              4   
       U                        61              61             61   
       W                         6               6              6   
Blue   U                        25              25             25   
Multi  M                         3               3              3   
       U                         5               5              5   
Red    U                        37              37             37   
Silver U                        22              22             22   
       W                         8               8              8   
White  U                         4               4              4   
Yello

What do we notice about this output? Are all columns the same? Why or why not?

We can see that the `.count()` method excludes nulls, and there is no way to change this with the current implementation:
```python
Signature: .count()
Docstring: Compute count of group, excluding missing values
```

As a best practice, you should either:
- fill in nulls prior to your .count(), or
- use the PK (primary key) of the table, which is guaranteed non-null

In [ ]:
prod.groupby(['Color']).count()['Name']

In [42]:
# Here we can use 'x' as a dummy placeholder for nulls, simply 
# to get consistent counts for all columns

#prod.fillna(value='x').groupby(['Color']).count()

prod.groupby(['Color']).count()

Column,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
Color,,,,,,,,,,,,,,,,,,,,,
Black,93,93,93,93,93,93,93,93,71,55,...,86,72,71,89,89,93,44,0,93,93
Blue,26,26,26,26,26,26,26,26,25,22,...,26,22,25,26,26,26,0,0,26,26
Grey,1,1,1,1,1,1,1,1,0,0,...,1,0,0,1,1,1,1,0,1,1
Multi,8,8,8,8,8,8,8,8,7,0,...,8,0,8,8,8,8,3,0,8,8
Red,38,38,38,38,38,38,38,38,37,37,...,38,37,37,38,38,38,30,0,38,38
Silver,43,43,43,43,43,43,43,43,31,30,...,31,30,30,36,36,43,6,0,43,43
Silver/Black,7,7,7,7,7,7,7,7,0,0,...,7,6,0,7,7,7,0,0,7,7
White,4,4,4,4,4,4,4,4,4,0,...,4,0,4,4,4,4,2,0,4,4
Yellow,36,36,36,36,36,36,36,36,36,32,...,36,32,36,36,36,36,0,0,36,36


Let's find out the most expensive price for an item, by `Color`:

In [49]:
prod.groupby(['Color'])['ListPrice'].max().sort_values(ascending=False)

Color
Red             3578.27
Silver          3399.99
Black           3374.99
Yellow          2384.07
Blue            2384.07
Grey             125.00
Multi             89.99
Silver/Black      80.99
White              9.50
Name: ListPrice, dtype: float64

We can also do multi-level groupbys. This is referred to as a `Multiindex` dataframe. Here, we can see the following fields in a nested group by, with a count of Name (with nulls filled!); effectively giving us a count of the number of products for every unique Class/Style combination:

- Class - H = High, M = Medium, L = Low
- Style - W = Womens, M = Mens, U = Universal

In [53]:
prod.groupby(['Class','Style']).count()['Name']

Class  Style
H      U        64
L      U        68
M      U        22
       W        22
Name: Name, dtype: int64

In [58]:
prod.groupby(['Class','Style']).count()

Column       Name  ProductNumber  MakeFlag  FinishedGoodsFlag  Color  \
Class Style                                                            
H     U        64             64        64                 64     64   
L     U        68             68        68                 68     68   
M     U        22             22        22                 22     22   
      W        22             22        22                 22     22   

Column       SafetyStockLevel  ReorderPoint  StandardCost  ListPrice  Size  \
Class Style                                                                  
H     U                    64            64            64         64    64   
L     U                    68            68            68         68    68   
M     U                    22            22            22         22    22   
      W                    22            22            22         22    22   

Column       ...  Weight  DaysToManufacture  ProductLine  \
Class Style  ...                                           
H     U      ...      64                 64           64   
L     U      ...      68                 68           68   
M     U      ...      22                 22           22   
      W      ...      22                 22           22   

Column       ProductSubcategoryID  ProductModelID  SellStartDate  SellEndDate  \
Class Style                                                                     
H     U                        64              64             64           20   
L     U                        68              68             68           18   
M     U                        22              22             22           18   
      W                        22              22             22            0   

Column       DiscontinuedDate  rowguid  ModifiedDate  
Class Style                                           
H     U                     0       64            64  
L     U                     0       68            68  
M     U                     0       22            22  
      W                     0       22            22  

[4 rows x 22 columns]

In [57]:
prod.groupby(['Class','Style']).mean()

Column       MakeFlag  FinishedGoodsFlag  SafetyStockLevel  ReorderPoint  \
Class Style                                                                
H     U           1.0                1.0        281.250000    210.937500   
L     U           1.0                1.0        288.235294    216.176471   
M     U           1.0                1.0        263.636364    197.727273   
      W           1.0                1.0        263.636364    197.727273   

Column       StandardCost    ListPrice     Weight  DaysToManufacture  \
Class Style                                                            
H     U       1247.776236  2106.310156  12.253906           2.671875   
L     U        302.935585   506.680588  14.416176           2.617647   
M     U        560.993659   947.204545  14.704091           2.818182   
      W        553.489518   905.221818  12.915909           2.863636   

Column       ProductSubcategoryID  ProductModelID  DiscontinuedDate  
Class Style                                                          
H     U                  7.281250       16.390625               NaN  
L     U                  7.647059       20.029412               NaN  
M     U                  6.545455       22.772727               NaN  
      W                  6.363636       22.090909               NaN

We can also use the `.agg()` method with multiple arguments, to simulate a `.describe()` method like we used before:

In [62]:
df=prod.groupby('Color')['ListPrice'].agg(['count','sum','mean','min','max'])

,count,sum,mean,min,max
Color,,,,,
Black,93,67436.26,725.121075,0.00,3374.99
Blue,26,24015.66,923.679231,34.99,2384.07
Grey,1,125.00,125.000000,125.00,125.00
Multi,8,478.92,59.865000,8.99,89.99
Red,38,53274.10,1401.950000,34.99,3578.27
Silver,43,36563.13,850.305349,0.00,3399.99
Silver/Black,7,448.13,64.018571,40.49,80.99
White,4,36.98,9.245000,8.99,9.50
Yellow,36,34527.29,959.091389,53.99,2384.07


In [69]:
df.groupby("group")['value'].mean()

NameError: name 'df' is not defined

In [ ]:
prod.fillna(value={'Color':"Missing"})\
.groupby(['color'])

## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, let's say we want to change our colors from a word, to just a single letter. How would we do that?

The first step is writing a function, with the argument being the value we would receive from each cell in the column. This function will mutate the input, and return the result. This result will then be _applied_ to the source dataframe (if desired).

In [70]:
# Get a list of all unique colors
prod['Color'].unique()

array([nan, 'Black', 'Silver', 'Red', 'White', 'Blue', 'Multi', 'Yellow',
       'Grey', 'Silver/Black'], dtype=object)

In [71]:
# Create a function to convert colors to letters
def color_to_letter(color): 
    color_dict = {
        'Black': 'B',
        'Silver': 'S',
        'Red': 'R',
        'White': 'W',
        'Blue': 'L',
        'Multi': 'M',
        'Yellow': 'Y',
        'Grey': 'G',
        'Silver/Black': 'V'
    }
    
    try:
        return color_dict[color]
    except:
        return 'N'

In [73]:
color_to_letter('Grey')

'G'

Now we can _apply_ this function to our `pd.Series` object, returning the result (which we can use to overwrite the source, if we choose).

In [75]:
prod['Color'].apply(color_to_letter).head(10)

ProductID
1      N
2      N
3      N
4      N
316    N
317    B
318    B
319    B
320    S
321    S
Name: Color, dtype: object

In [76]:
prod['Color'].apply(color_to_letter).value_counts()

N    248
B     93
S     43
R     38
Y     36
L     26
M      8
V      7
W      4
G      1
Name: Color, dtype: int64

The `pd.DataFrame.apply` implementation is similar, however it effectively 'scrolls through' the columns and passes each one sequentially to your function:

```python
Objects passed to the function are Series objects whose index is
either the DataFrame index (axis=0) or the DataFrame columns (axis=1).
```

It should only be used when you wish to apply the same function to all columns (or rows) of your `pd.DataFrame` object.

We can also use `pd.Series.apply()` with a **lambda expression**. This is an undeclared function and is commonly used for simple functions within the `.apply()` method. Let's use it to add $100 to our `ListPrice` column. Hey, baby needs new shoes!

In [79]:
# Without apply - using a lambda

prod['ListPrice'] +100

ProductID
1      100.00
2      100.00
3      100.00
4      100.00
316    100.00
        ...  
995    201.24
996    221.49
997    639.99
998    639.99
999    639.99
Name: ListPrice, Length: 504, dtype: float64

In [78]:
# And now with 100 more dollars!.apply(lambda x:X+100)

prod['ListPrice'].apply(lambda x:x+100)

ProductID
1      100.00
2      100.00
3      100.00
4      100.00
316    100.00
        ...  
995    201.24
996    221.49
997    639.99
998    639.99
999    639.99
Name: ListPrice, Length: 504, dtype: float64

Boom! Maybe financing that new boat wasn't such a bad idea after all!

**Your turn:** Identify one other column where we may want to write a new apply function, or use the one we just created for the purposes of cleaning up our dataset.

In [85]:
# Identify a column to mutate (change)
prod['StandardCost']

ProductID
1        0.0000
2        0.0000
3        0.0000
4        0.0000
316      0.0000
         ...   
995     44.9506
996     53.9416
997    343.6496
998    343.6496
999    343.6496
Name: StandardCost, Length: 504, dtype: float64

In [87]:
prod['StandardCost'].isnull().sum()

0

In [84]:
# write a function to mutate that column (or columns) 
# note: if using a lambda function, you can leave this blank
prod['StandardCost'].apply(lambda x:x+100)

ProductID
1      100.0000
2      100.0000
3      100.0000
4      100.0000
316    100.0000
         ...   
995    144.9506
996    153.9416
997    443.6496
998    443.6496
999    443.6496
Name: StandardCost, Length: 504, dtype: float64

In [88]:
# Apply that function across the whole column
prod['StandardCost'].apply(lambda x:x+10).round(2)

ProductID
1       10.00
2       10.00
3       10.00
4       10.00
316     10.00
        ...  
995     54.95
996     63.94
997    353.65
998    353.65
999    353.65
Name: StandardCost, Length: 504, dtype: float64

In [90]:
prod['SellStartDate'].apply(lambda x: x[:4])

ProductID
1      2008
2      2008
3      2008
4      2008
316    2008
       ... 
995    2013
996    2013
997    2013
998    2013
999    2013
Name: SellStartDate, Length: 504, dtype: object

---
<h1>Joining Table with Pandas</h1>

Pandas provides support for combining `Series`, `DataFrame` and even `xarray` (3 dimensional `DataFrame`s, formerly known in pandas v0.20.0 as `Panel`s) objects with various kinds of set logic for the indicies and relational algebra functionality in the case of join / merge-type operations. More simply stated, this allows you to combine `DataFrame`s.

<!-- Overview -->
<details>
    <summary>Overview</summary>
    <ul>
        <li><b>In this session, we'll cover:</b></li>
        <br>
        <ul>
            <li>Concatenating objects with <code>.append()</code> and <code>.concat()</code></li>
            <li>Combining objects with <code>.join()</code> and <code>.merge()</code></li>
            <li>Combining timeseries objects with <code>.merge_ordered()</code></li>
            <li>Traditionally, this functionality is performed in a relational database, such as <a href="https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html#compare-with-sql-join">SQL</a>. With pandas, you'll be able to perform the same operations - in python! The backend is <code>numpy</code>, a powerful linear algebra library which helps keep things speedy</li>
        </ul>
        <br>
        <li><b>Why Join?</b></li>
        <br>
        <ul>
            <li>You might be asking yourself - why keep data separated in different files? <i>Why not just keep it all in one file?</i></li>
            <li>The answer stems from a thing called <a href="https://support.microsoft.com/en-us/help/283878/description-of-the-database-normalization-basics">database normalization</a>. When a database is <i>normalized</i>, it is structured in such a way that redundancy of data is minimized. This allows a database to be faster, smaller, and more flexible when it comes time to change the data inside of it</li>
            <li>The manifestation of this <i>normalization</i> is data that is represented within multiple <a href="https://en.wikipedia.org/wiki/Table_(database)">tables</a> (which are effectively dataframes), related to each other by <a href="https://www.studytonight.com/dbms/database-key.php">keys</a>, or columns in one table that equal a column in another table, allowing them to be joined. In this case, our tables are the <code>.csv</code> files we'll be importing</li>
        </ul>
    </ul>
</details>

<!-- TOC -->
<details>
    <summary>Table of Contents</summary>
    <ul>
        <li><a href="#conapp">Concatenate and Append</a></li>
        <ul>
            <li><a href="#concatenate">Concatenate</a></li>
            <li><a href="#append">Append</a></li>
        </ul>
        <li><a href="#joining">Joining</a></li>
        <ul>
            <li><a href="join">Join</a></li>
            <li><a href="#merge">Merge</a></li>
            <ul>
                <li><a href="#merge_keycols">Merge on Non-Index Columns</a></li>
                <li><a href="#yourturn">Now it's Your Turn!</a></li>
            </ul>
        </ul>
        <li><a href="#exercise">Exercise - AdventureWorks</a></li>
        <ul>
            <li><a href="#p_exercise">Table Joins on Live Data</a></li>
            <ul>
                <li><a href="#ex_pp">Join Product Tables</a></li>
                <li><a href="#ex_soh_sod">Join Sales Order Header and Sales Order Detail Tables</a></li>
                <li><a href="#ex_soh_sod_pt">Join Sales Order Header, Sales Order Detail, and Product Tables</a></li>
            </ul>
        </ul>
    </ul>
</details>

<div id="conapp"></div>
<h2>Concatenate and Append</h2>

<div id="concatenate"></div>
<h3>Concatenate</h3>

Concatenate sticks dataframes together, either on top of each other, or next to each other.

```python
Signature: pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
Docstring:
Concatenate pandas objects along a particular axis with optional set logic
along the other axes.
```

First, let's create two dataframes, `df1` and `df2`.

In [91]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df1.head()

,letter,number
0,a,1
1,b,2


In [92]:
# KEEP
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])
df2.head()

,letter,number
0,c,3
1,d,4


Next, let's stick the dataframes on top of each other using `concat`. 

In [96]:
pd.concat([df1,df2],ignore_index=True)

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [97]:
pd.concat([df1,df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


Finally, let's stick the dataframes <b>next</b> to each other using `concat`. Use of the `axis` kwarg will help us here.

In [95]:
pd.concat([df1,df2],axis=1)

,letter,number,letter,number
0,a,1,c,3
1,b,2,d,4


<div id="append"></div>
<h3>Append</h3>

Append is very similar to `concat`, except it limits itself to a specific case of `concat`, where `axis=0` (stack on top of each other) and `join=outer` (how to handle the axis of the second dataframe). For almost all cases, `concat` has all the functionality of `append` (and more) and can replace `append` entirely.

```python
Signature: pd.DataFrame.append(self, other, ignore_index=False, verify_integrity=False, sort=None)
Docstring:
Append rows of `other` to the end of this frame, returning a new
object. Columns not in this frame are added as new columns.
```

Also note that `append` is a DataFrame and Series method, and not a pandas library function like `concat` is.

<div id="joining"></div>
<h2>Joining</h2>

<div id="join"></div>
<h3>Join</h3>

`join` allows us to compare two dataframes, and combine them by using a matching column known as a `key`. Normally, during joins, this key is explicitly stated (we'll get to this with `merge` in our next example). With `join`, the `key` joining the table is always the `index` of the first table with (by default) the index of the second table. 

```python
Signature: pd.DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
Docstring:
Join columns with other DataFrame either on index or on a key
column. Efficiently Join multiple DataFrame objects by index at once by
passing a list.
```

First, let's create two dataframes.

In [98]:
# KEEP
df1 = pd.DataFrame([['a', 1], ['b', 2], ['c', 3], ['d', 4]], columns=['letter', 'number'])
df1.head()

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [99]:
# KEEP
df2 = pd.DataFrame([['e', 5], ['f', 6]], columns=['letter', 'number'])
df2.head()

,letter,number
0,e,5
1,f,6


In [106]:
df1

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [107]:
df2

,letter,number
0,e,5
1,f,6


Now, lets `join` these two dataframes. Note that we will `key`, or 'line up', the two dataframes based on their `indicies`.

Note that, when joining dataframes with any common column names, we will need to supply a `lsuffix` or `rsuffix` kwarg. This is appended to the end of the column name of the returned, joined dataframe to differentiate and identify the source column. Here, we'll use `_df1` to identify that the column shown came from the `df1` dataframe, and `_df2` as a suffix to identify its origin as the `df2` dataframe. 

In [111]:
df1.join(df2,how='left',lsuffix='_df1',rsuffix='_df2')

,letter_df1,number_df1,letter_df2,number_df2
0,a,1,e,5.0
1,b,2,f,6.0
2,c,3,NaN,NaN
3,d,4,NaN,NaN


Note how we have joined the two dataframes on their indicies, which creates a null for rows of index 2 and 3 in `df2`. This is expected and correct.

Also note that the default join behavior of `join` is `left`. We can change this with the `how` kwarg.

For reference, here are the common types of joins. Join types won't be covered in this lesson.
<p align="center">
<img width="500px" src="https://i.stack.imgur.com/udQpD.jpg">
</p>

The type of join we performed above is shown in the upper-left most figure in the above chart.

<div id="merge"></div>
<h3>Merge</h3>

Similar to `join` is `merge`. The difference between the two is the <i>keying behavior</i>. `merge` has a richer API (more functionality) and allows one to join on columns in the source dataframe <i>other than the index</i>. Because `merge` can effectively do everything that `join` can do, and more - it is recommended to always use `merge` unless code brevity is the top concern. 

```python
Signature: pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
Docstring:
Merge DataFrame objects by performing a database-style join operation by
columns or indexes.
```

Note that `merge` is <i>both</i> a DataFrame method as well as a pandas function. Below, we'll be using the pandas function, `pd.merge()`.

In [112]:
print(df1)
print(df2)

  letter  number
0      a       1
1      b       2
2      c       3
3      d       4
  letter  number
0      e       5
1      f       6


In [119]:
pd.merge(df1,df2,how='left',left_index=True,right_index=True,suffixes=('_df1','_df2'))

,letter_df1,number_df1,letter_df2,number_df2
0,a,1,e,5.0
1,b,2,f,6.0
2,c,3,NaN,NaN
3,d,4,NaN,NaN


In [118]:
pd.merge(df1,df2,how='left')

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


Note that we've achieved the same exact output as we did with `join`, but it took a little more explicit work. Let's run through the arguments for clarity:

<ul>
    <li><code>df1</code>: this is the first dataframe, and considered to be on the 'left' of <code>df2</code></li>
    <li><code>df2</code>: this is the second dataframe, considered to be on the right of <code>df1</code></li>
    <li><code>how='left'</code>: this states the type of join; see the above SQL join table</li>
    <li><code>left_index=True</code>: this uses the index of <code>df1</code> as the join key for the left table</li>
    <li><code>right_index=True</code>: this uses the index of <code>df2</code> as the join key for the right table</li>
    <li><code>suffixes</code>: this places <code>_df1</code> after column names which came from <code>df1</code></li>
</ul>

<div id="merge_keycols"></div>
<h4>Merge on Non-Index Columns</h4>

This brings us to our next point: merging on columns that are not the index columns. This is very, very common in SQL joins and this technique can be used to join just about any DataFrame.

First, let's create some more realistic data - stocks!

In [115]:
# KEEP
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 
                          'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], 
                       '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 
                          'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})

Now, let's join the <code>openprice</code> and <code>wkhigh</code> dataframes together.

In [117]:
pd.merge(openprice,wkhigh,how='left')

,Symbol,OpenPrice,52wkHigh
0,AAPL,217.51,233.47
1,DHR,96.54,110.11
2,DAL,51.45,60.79
3,AMZN,1703.34,2050.49


Note how our `Symbol` column isn't in the same order in each dataframe. This is intentional, and note that the dataframe on the left, `openprice` dictates the order of the dataframe on the right, `wkhigh`. Also note that the shared key between the two dataframes is exempt from having a <code>suffix</code> applied to it. 

<div id="yourturn"></div>
<h4>Now it's your turn!</h4>

<ul>
    <li><code>merge</code> the <code>openprice</code> and <code>stockname</code> dataframes and inspect the result</li>
    <li><code>merge</code> all three dataframes together and inspect the result</li>
</ul>

In [124]:
openprice2=pd.merge(openprice,stockname,how='inner',left_on='Symbol',right_on='Symbol')

In [123]:
openprice2=pd.merge(openprice,stockname,how='inner')

,Symbol,OpenPrice,Name
0,AAPL,217.51,Apple
1,DHR,96.54,Danaher
2,DAL,51.45,Delta Airlines
3,AMZN,1703.34,Amazon


In [127]:
pd.merge(openprice,stockname,how='inner',left_on='Symbol',right_on='Symbol').merge(wkhigh,on='Symbol')

,Symbol,OpenPrice,Name,52wkHigh
0,AAPL,217.51,Apple,233.47
1,DHR,96.54,Danaher,110.11
2,DAL,51.45,Delta Airlines,60.79
3,AMZN,1703.34,Amazon,2050.49


<div id="exercise"></div>
<h2>Exercise - Adventure Works</h2>
<p align="right">
<img src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">
</p>

<div id="p_exercise"></div>
<h3>Table Joins on Live Data</h3>

Here are the data dictionaries we'll be using for the following exercise:

<ul>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html">Production.Product</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Production.ProductSubCategory.html">Production.ProductSubcategory</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderHeader.html">Sales.SalesOrderHeader</a></li>
    <li><a href="https://www.sqldatadictionary.com/AdventureWorks2014/Sales.SalesOrderDetail.html">Sales.SalesOrderDetail</a></li>
</ul>

In [128]:
p = pd.read_csv('Production.Product.csv', sep='\t')
ps = pd.read_csv('Production.ProductSubcategory.csv', sep='\t')
soh = pd.read_csv('Sales.SalesOrderHeader.csv', sep='\t', nrows=1000)
sod = pd.read_csv('Sales.SalesOrderDetail.csv', sep='\t', nrows=1000)

<div id="ex_pp"></div>
<h4>Join Product Tables</h4>

<ul>
    <li>Using the <code>Production.Product.ProductID</code> and <code>Production.ProductSubcategory.ProductID</code> keys, join the <code>Production.Product</code> and <code>Production.ProductSubcategory</code> tables</li>
</ul>

In [135]:
p.columns

Index(['ProductID', 'Name', 'ProductNumber', 'MakeFlag', 'FinishedGoodsFlag',
       'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost',
       'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode',
       'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style',
       'ProductSubcategoryID', 'ProductModelID', 'SellStartDate',
       'SellEndDate', 'DiscontinuedDate', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [136]:
ps.columns

Index(['ProductSubcategoryID', 'ProductCategoryID', 'Name', 'rowguid',
       'ModifiedDate'],
      dtype='object')

In [139]:
pd.merge(p,ps,how='inner',on='ProductSubcategoryID')

,ProductID,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid_x,ModifiedDate_x,ProductCategoryID,Name_y,rowguid_y,ModifiedDate_y
0,680,"HL Road Frame - Black, 58",FR-R92B-58,1,1,Black,500,375,1059.3100,1431.50,...,6.0,2008-04-30 00:00:00,NaN,NaN,{43DD68D6-14A4-461F-9069-55309D90EA7E},2014-02-08 10:01:36.827000000,2,Road Frames,{5515F857-075B-4F9A-87B7-43B4997077B3},2008-04-30 00:00:00
1,706,"HL Road Frame - Red, 58",FR-R92R-58,1,1,Red,500,375,1059.3100,1431.50,...,6.0,2008-04-30 00:00:00,NaN,NaN,{9540FF17-2712-4C90-A3D1-8CE5568B2462},2014-02-08 10:01:36.827000000,2,Road Frames,{5515F857-075B-4F9A-87B7-43B4997077B3},2008-04-30 00:00:00
2,717,"HL Road Frame - Red, 62",FR-R92R-62,1,1,Red,500,375,868.6342,1431.50,...,6.0,2011-05-31 00:00:00,NaN,NaN,{052E4F8B-0A2A-46B2-9F42-10FEBCFAE416},2014-02-08 10:01:36.827000000,2,Road Frames,{5515F857-075B-4F9A-87B7-43B4997077B3},2008-04-30 00:00:00
3,718,"HL Road Frame - Red, 44",FR-R92R-44,1,1,Red,500,375,868.6342,1431.50,...,6.0,2011-05-31 00:00:00,NaN,NaN,{A88D3B54-2CAE-43F2-8C6E-EA1D97B46A7C},2014-02-08 10:01:36.827000000,2,Road Frames,{5515F857-075B-4F9A-87B7-43B4997077B3},2008-04-30 00:00:00
4,719,"HL Road Frame - Red, 48",FR-R92R-48,1,1,Red,500,375,868.6342,1431.50,...,6.0,2011-05-31 00:00:00,NaN,NaN,{07BEFC9A-7634-402B-B234-D7797733BAAF},2014-02-08 10:01:36.827000000,2,Road Frames,{5515F857-075B-4F9A-87B7-43B4997077B3},2008-04-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,978,"Touring-3000 Blue, 44",BK-T18U-44,1,1,Blue,100,75,461.4448,742.35,...,36.0,2013-05-30 00:00:00,NaN,NaN,{4F638E15-2ED0-4193-90CE-47DA580E01DD},2014-02-08 10:01:36.827000000,1,Touring Bikes,{02C5061D-ECDC-4274-B5F1-E91D76BC3F37},2008-04-30 00:00:00
291,979,"Touring-3000 Blue, 50",BK-T18U-50,1,1,Blue,100,75,461.4448,742.35,...,36.0,2013-05-30 00:00:00,NaN,NaN,{DE305B62-88FC-465B-B23D-D8C0F7E6D361},2014-02-08 10:01:36.827000000,1,Touring Bikes,{02C5061D-ECDC-4274-B5F1-E91D76BC3F37},2008-04-30 00:00:00
292,994,LL Bottom Bracket,BB-7421,1,1,NaN,500,375,23.9716,53.99,...,95.0,2013-05-30 00:00:00,NaN,NaN,{FA3C65CD-0A22-47E3-BDF6-53F1DC138C43},2014-02-08 10:01:36.827000000,2,Bottom Brackets,{A9E54089-8A1E-4CF5-8646-E3801F685934},2008-04-30 00:00:00
293,995,ML Bottom Bracket,BB-8107,1,1,NaN,500,375,44.9506,101.24,...,96.0,2013-05-30 00:00:00,NaN,NaN,{71AB847F-D091-42D6-B735-7B0C2D82FC84},2014-02-08 10:01:36.827000000,2,Bottom Brackets,{A9E54089-8A1E-4CF5-8646-E3801F685934},2008-04-30 00:00:00


<div id="ex_soh_sod"></div>
<h4>Join Sales Order Header and Sales Order Detail Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code> and <code>Sales.SalesOrderDetail</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>

In [140]:
soh.columns

Index(['SalesOrderID', 'RevisionNumber', 'OrderDate', 'DueDate', 'ShipDate',
       'Status', 'OnlineOrderFlag', 'SalesOrderNumber', 'PurchaseOrderNumber',
       'AccountNumber', 'CustomerID', 'SalesPersonID', 'TerritoryID',
       'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 'CreditCardID',
       'CreditCardApprovalCode', 'CurrencyRateID', 'SubTotal', 'TaxAmt',
       'Freight', 'TotalDue', 'Comment', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [141]:
sod.columns

Index(['SalesOrderID', 'SalesOrderDetailID', 'CarrierTrackingNumber',
       'OrderQty', 'ProductID', 'SpecialOfferID', 'UnitPrice',
       'UnitPriceDiscount', 'LineTotal', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [144]:
pd.merge(soh,sod,how='inner',on='SalesOrderID',validate='1:m')

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid_y,ModifiedDate_y
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,1,4911-403C-98,1,776,1,2024.9940,0.0,2024.9940,{B207C96D-D9E6-402B-8470-2CC176C42283},2011-05-31 00:00:00
1,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,2,4911-403C-98,3,777,1,2024.9940,0.0,6074.9820,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},2011-05-31 00:00:00
2,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,3,4911-403C-98,1,778,1,2024.9940,0.0,2024.9940,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},2011-05-31 00:00:00
3,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,4,4911-403C-98,1,771,1,2039.9940,0.0,2039.9940,{04C4DE91-5815-45D6-8670-F462719FBCE3},2011-05-31 00:00:00
4,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,5,4911-403C-98,1,772,1,2039.9940,0.0,2039.9940,{5A74C7D2-E641-438E-A7AC-37BF23280301},2011-05-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,43892,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43892,PO2523117473,10-4020-000167,...,996,0D3C-4E1F-9C,2,765,1,419.4589,0.0,838.9178,{30EDDEA5-2C93-4F08-B66E-BFADB84AFF0E},2011-07-01 00:00:00
996,43893,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43893,PO2204129382,10-4020-000370,...,997,7C5D-46AA-A7,1,770,1,419.4589,0.0,419.4589,{46A1DA27-B11D-4918-8308-4B7715C12FB5},2011-07-01 00:00:00
997,43894,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43894,PO2958194987,10-4020-000559,...,998,7DF2-4E54-B0,3,710,1,5.7000,0.0,17.1000,{6005BBFE-CD7C-45FD-80F7-DBBADB10BFEC},2011-07-01 00:00:00
998,43894,8,2011-07-01 00:00:00,2011-07-13 00:00:00,2011-07-08 00:00:00,5,0,SO43894,PO2958194987,10-4020-000559,...,999,7DF2-4E54-B0,2,777,1,2024.9940,0.0,4049.9880,{03FF934E-043F-4CE7-8F44-18BFEE7EDBB7},2011-07-01 00:00:00


<div id="ex_soh_sod_pt"></div>
<h4>Join Sales Order Header, Sales Order Detail, and Product Tables</h4>

<ul>
    <li>Join the <code>Sales.SalesOrderHeader</code>, <code>Sales.SalesOrderDetail</code>, and <code>Production.Product</code> tables</li>
    <li>Don't forget to use your data dictionaries!</li>
</ul>

In [146]:
pd.merge(soh,sod,how='inner',on='SalesOrderID',validate='1:m').\
merge(p,how='inner',on='ProductID')

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,43659,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43659,PO522145787,10-4020-000676,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
1,43661,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43661,PO18473189620,10-4020-000442,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
2,43665,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43665,PO16588191572,10-4020-000146,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
3,43670,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43670,PO14384116310,10-4020-000504,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
4,43672,8,2011-05-31 00:00:00,2011-06-12 00:00:00,2011-06-07 00:00:00,5,0,SO43672,PO13862153537,10-4020-000119,...,M,H,U,1.0,19.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{02935111-A546-4C6D-941F-BE12D42C158E},2014-02-08 10:01:36.827000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,43774,8,2011-06-18 00:00:00,2011-06-30 00:00:00,2011-06-25 00:00:00,5,1,SO43774,NaN,10-4030-016348,...,R,H,U,2.0,25.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{5E085BA0-3CD5-487F-85BB-79ED1C701F23},2014-02-08 10:01:36.827000000
996,43808,8,2011-06-25 00:00:00,2011-07-07 00:00:00,2011-07-02 00:00:00,5,1,SO43808,NaN,10-4030-013259,...,R,H,U,2.0,25.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{5E085BA0-3CD5-487F-85BB-79ED1C701F23},2014-02-08 10:01:36.827000000
997,43822,8,2011-06-27 00:00:00,2011-07-09 00:00:00,2011-07-04 00:00:00,5,1,SO43822,NaN,10-4030-027669,...,R,H,U,2.0,25.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{5E085BA0-3CD5-487F-85BB-79ED1C701F23},2014-02-08 10:01:36.827000000
998,43825,8,2011-06-27 00:00:00,2011-07-09 00:00:00,2011-07-04 00:00:00,5,1,SO43825,NaN,10-4030-016349,...,R,H,U,2.0,25.0,2011-05-31 00:00:00,2012-05-29 00:00:00,NaN,{5E085BA0-3CD5-487F-85BB-79ED1C701F23},2014-02-08 10:01:36.827000000
